# XAUUSD LSTM AutoLoad
مدل LSTM برای پیش‌بینی طلا

In [ ]:
!pip install pandas numpy scikit-learn tensorflow matplotlib

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

In [ ]:
# بارگذاری دیتای CSV
df = pd.read_csv('xauusd_lstm_ready_clean.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.dropna(inplace=True)
df.head()

In [ ]:
# آماده‌سازی داده برای مدل LSTM
features = ['close', 'rsi', 'atr', 'ema_diff', 'body', 'upper_shadow', 'lower_shadow']
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])
X, y = [], []
seq_len = 60
for i in range(seq_len, len(scaled) - 10):
    X.append(scaled[i-seq_len:i])
    future = df['close'].iloc[i + 10]
    now = df['close'].iloc[i]
    delta = future - now
    threshold = df['atr'].iloc[i] * 1.5
    if delta > threshold:
        y.append(2)
    elif delta < -threshold:
        y.append(0)
    else:
        y.append(1)
X, y = np.array(X), np.array(y)

In [ ]:
# ساخت و آموزش مدل
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc:.4f}')